In [ ]:
import prometheus_api_client as pac
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timezone, timedelta
from dateutil import tz
from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
from numpy import arange
from matplotlib.ticker import FormatStrFormatter

In [ ]:
prom = pac.PrometheusConnect(url="http://localhost:8069", disable_ssl=True)

In [ ]:
def create_report(target_metric_list, title, start, end, step, tick_locator, tick_label_formatter):
    fig, axs = plt.subplots(len(target_metric_list), 1, figsize=(15, len(target_metric_list) * 10))
    plt.suptitle(title)
    plt.tight_layout(h_pad=5, rect=[0, 0, 1, 0.97]) 
    for i, target_metric in enumerate(target_metric_list):
        metric_query = target_metric['query']
        metric_data = prom.custom_query_range(
            query=metric_query,
            start_time=start,
            end_time=end,
            step=step
        )
        legends = []
        ax = axs[i]
        metric_data = sorted(metric_data, key=lambda d: sum(float(value[1]) for value in d['values']) / len(d['values']), reverse=True)
        if 'max_num' in target_metric:
            metric_data = metric_data[:target_metric['max_num']]
        for metric_datum in metric_data:
            label = ', '.join([f'{pair[0]}: {pair[1]}'  for pair in metric_datum['metric'].items() if pair[0] != '__name__'])
            xdata = []
            ydata = []
            for point in metric_datum['values']:
                xdata.append(datetime.fromtimestamp(point[0]))
                ydata.append(float(point[1]) * target_metric['coefficient'])
            ax.plot_date(np.array(xdata), np.array(ydata), '-')
            ax.set_xlim(start, end)
            if 'range' in target_metric:
                ax.set_ylim(target_metric['range'][0], target_metric['range'][1])

            ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f ' + target_metric['unit']))
            ax.xaxis.set_major_locator(tick_locator)
            ax.xaxis.set_major_formatter(tick_label_formatter)
            legends.append(label)
        if(len(legends) > 1):
            ax.legend(labels=legends)
        if 'title' in target_metric:
            ax.set_title(target_metric['title'])
        else:
            ax.set_title(metric_query)